In [3]:
# pip install opencv-python numpy pillow tk tqdm

In [5]:
import cv2
import numpy as np
import tkinter as tk
import os
from tkinter import filedialog, Label, Button, Canvas, ttk
from PIL import Image, ImageTk
import time

class CartoonizerApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Cartoonizer App")
        self.root.geometry("600x800")

        self.img_path = None
        self.original_image = None
        self.cartoon_image = None

        # UI Components
        self.label = Label(root, text="Cartoon Image Converter", font=("Arial", 16))
        self.label.pack(pady=10)

        self.canvas = Canvas(root, width=500, height=400, bg="gray")
        self.canvas.pack()

        self.btn_select = Button(root, text="Select Image", command=self.load_image)
        self.btn_select.pack(pady=5)

        self.btn_cartoon = Button(root, text="Apply Cartoon Effect", command=self.apply_cartoon)
        self.btn_cartoon.pack(pady=5)

        self.progress = ttk.Progressbar(root, orient="horizontal", length=300, mode="determinate")
        self.progress.pack(pady=10)

        self.btn_save = Button(root, text="Save Image", command=self.save_image)
        self.btn_save.pack(pady=5)

        self.btn_download = Button(root, text="Download to Root Folder", command=self.download_image)
        self.btn_download.pack(pady=5)

    def load_image(self):
        file_path = filedialog.askopenfilename(filetypes=[("Image Files", "*.jpg *.png *.jpeg")])
        if not file_path:
            return
        self.img_path = file_path
        self.original_image = cv2.imread(file_path)
        self.display_image(self.original_image)

    def apply_cartoon(self):
        if self.original_image is None:
            return

        self.progress["value"] = 0
        self.root.update_idletasks()

        time.sleep(0.5)  # Simulating processing time

        # Convert image to grayscale
        gray = cv2.cvtColor(self.original_image, cv2.COLOR_BGR2GRAY)
        gray = cv2.medianBlur(gray, 7)  # Increased blur for a smoother effect

        self.progress["value"] = 30
        self.root.update_idletasks()

        # Detect edges with adaptive threshold
        edges = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 11, 11)

        self.progress["value"] = 50
        self.root.update_idletasks()

        # Apply stronger bilateral filter for a more defined cartoon effect
        color = cv2.bilateralFilter(self.original_image, 15, 300, 300)  # Increased filter strength

        self.progress["value"] = 80
        self.root.update_idletasks()

        # Combine edges and color
        cartoon = cv2.bitwise_and(color, color, mask=edges)
        self.cartoon_image = cartoon

        self.progress["value"] = 100
        self.root.update_idletasks()

        self.display_image(self.cartoon_image)

    def display_image(self, img):
        """ Display a scaled-down preview while keeping the original size for processing. """
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img_pil = Image.fromarray(img_rgb)

        # Resize for preview but keep the original image untouched
        img_preview = img_pil.copy()
        img_preview.thumbnail((500, 400))

        img_tk = ImageTk.PhotoImage(img_preview)
        self.canvas.create_image(250, 200, image=img_tk, anchor=tk.CENTER)
        self.canvas.image = img_tk  # Keep a reference to avoid garbage collection

    def save_image(self):
        if self.cartoon_image is None:
            return

        save_path = filedialog.asksaveasfilename(defaultextension=".png",
                                                 filetypes=[("PNG files", "*.png"),
                                                            ("JPEG files", "*.jpg"),
                                                            ("All Files", "*.*")])
        if save_path:
            cv2.imwrite(save_path, self.cartoon_image)
            print(f"Image saved successfully at {save_path}")

    def download_image(self):
        """ Saves the cartoonized image directly to the root folder with a default name. """
        if self.cartoon_image is None:
            return

        root_folder_path = os.path.join(os.getcwd(), "cartoon_image.png")  # Save in the same folder as script
        cv2.imwrite(root_folder_path, self.cartoon_image)
        print(f"Image downloaded to root folder: {root_folder_path}")

if __name__ == "__main__":
    root = tk.Tk()
    app = CartoonizerApp(root)
    root.mainloop()